# Combine the idSeq output files

Steps:

1. read in files
2. add nt mapping and nr mapping reads
3. combine samples into one table [outside merge]
4. reduce to all not eukaryota
5. save

## 1: Read in files (and load packages)

In [2]:
# !pip install pandas
import pandas as pd
import numpy as np
from functools import reduce
import glob 
import os

In [3]:
path_orig = 'C:\\Users\\Seth\\Documents\\GitHub\\JemimaWestern\\westernIDseq\\jemimamasterssb_1454\\'
# path_orig = './jemimamasterssb_1454/' # online

all_files_orig = glob.glob(path_orig + "*.csv")
# all_files_orig


In [4]:
ids = []
# extract sample id from the file name
def getIds(x):
    id = file.split('\\')[-1].split('_')[0] # '/' for online
    return id

for file in all_files_orig:
    ids.append(getIds(file))

print(ids)


['SRR11440601', 'SRR11440602', 'SRR11440603', 'SRR11440604', 'SRR11440605', 'SRR11440606', 'SRR11440607', 'SRR11440608', 'SRR11440609', 'SRR11440610', 'SRR11440611', 'SRR11440612', 'SRR11440613', 'SRR11440614', 'SRR11440615', 'SRR11440616', 'SRR11440617', 'SRR11440618', 'SRR11440619', 'SRR11440620', 'SRR11440621', 'SRR11440622', 'SRR11440623', 'SRR11440624', 'SRR11440625', 'SRR11440626', 'SRR11440627', 'SRR11440628', 'SRR11440629', 'SRR11440630', 'SRR11440631', 'SRR11440632', 'SRR11440633', 'SRR11440634', 'SRR11440635', 'SRR11440636', 'SRR11440637', 'SRR11440638', 'SRR11440639', 'SRR11440640', 'SRR11440641', 'SRR11440642', 'SRR11440643', 'SRR11440644', 'SRR11440645', 'SRR11440646', 'SRR11440647', 'SRR11440648', 'SRR11440649', 'SRR11440650', 'SRR11440651', 'SRR11440652', 'SRR11440653', 'SRR11440654', 'SRR11440655', 'SRR11440656', 'SRR11440657', 'SRR11440658', 'SRR11440659', 'SRR11440660', 'SRR11440661', 'SRR11440662', 'SRR11440663', 'SRR11440664', 'SRR11440665', 'SRR11440666', 'SRR11440

In [5]:

# Make empty dictionaries to store files in
dfs_agg ={} # agg is the aggregate score (prob wont use)

# test names make sense
# for f in all_files_orig:
    # print(os.path.splitext(os.path.basename(f))[0].split('_')[0])

#  read files into dictionary
for f in all_files_orig:
    dfs_agg[os.path.splitext(os.path.basename(f))[0].split('_')[0]] =  pd.read_csv(f, sep=',', header=0, usecols=["tax_id",	"tax_level",'genus_tax_id',	'name',	'category',"agg_score"])


dfs_nrnt ={} #nr nt is more likely what we'll use
for f in all_files_orig:
    dfs_nrnt[os.path.splitext(os.path.basename(f))[0].split('_')[0]] = pd.read_csv(f, sep=',', header=0, usecols=["tax_id",	"tax_level", 'genus_tax_id','name',	'category', "nt_count", "nr_count"])

# dfs_agg['SRR11440668']

dfs_nrnt['SRR11440668']

,tax_id,tax_level,genus_tax_id,name,category,nt_count,nr_count
0,1980421,2,1980421,Phasivirus,viruses,892,633.0
1,1980610,1,1980421,Phasi Charoen-like phasivirus,viruses,892,633.0
2,1884455,2,1884455,Anphevirus,viruses,190,124.0
3,2230910,1,1884455,Aedes anphevirus,viruses,170,123.0
4,2059675,1,1884455,Aedes aegypti anphevirus,viruses,20,1.0
...,...,...,...,...,...,...,...
107,1747,1,1912216,Cutibacterium acnes,bacteria,22,0.0
108,561,2,561,Escherichia,bacteria,20,0.0
109,562,1,561,Escherichia coli,bacteria,20,0.0
110,2282523,2,2282523,Methylorubrum,bacteria,2,NaN


In [6]:
# Need to make a master file with all tax info. 

tax = pd.concat(dfs_nrnt.values(), ignore_index=True)
tax = tax[['tax_id', 'tax_level', 'genus_tax_id', 'name', 'category']]  # just the taxa info

# tax 12707 rows
# get unique

tax=tax.drop_duplicates()

# now 2444 taxa
tax.head()
# dfs_nrnt.head()

,tax_id,tax_level,genus_tax_id,name,category
0,1980421,2,1980421,Phasivirus,viruses
1,1980610,1,1980421,Phasi Charoen-like phasivirus,viruses
2,-200011006,2,-200011006,non-genus-specific reads in family Totiviridae,viruses
3,2603435,1,-200011006,Aedes aegypti totivirus,viruses
4,1884455,2,1884455,Anphevirus,viruses


## 2. Calculate the total number of counts (both nt and nr) 


In [7]:
for key in dfs_nrnt: # add nr counts and nt counts, may be prob w nan's forcing to be float
    dfs_nrnt[key]['totCounts']=dfs_nrnt[key]['nt_count']+dfs_nrnt[key]['nr_count']
dfs_nrnt['SRR11440668'].head()

,tax_id,tax_level,genus_tax_id,name,category,nt_count,nr_count,totCounts
0,1980421,2,1980421,Phasivirus,viruses,892,633.0,1525.0
1,1980610,1,1980421,Phasi Charoen-like phasivirus,viruses,892,633.0,1525.0
2,1884455,2,1884455,Anphevirus,viruses,190,124.0,314.0
3,2230910,1,1884455,Aedes anphevirus,viruses,170,123.0,293.0
4,2059675,1,1884455,Aedes aegypti anphevirus,viruses,20,1.0,21.0


In [8]:
# make a df with just the taxa info
df_merged_cnts=pd.DataFrame(tax)
df_merged_cnts.head()


,tax_id,tax_level,genus_tax_id,name,category
0,1980421,2,1980421,Phasivirus,viruses
1,1980610,1,1980421,Phasi Charoen-like phasivirus,viruses
2,-200011006,2,-200011006,non-genus-specific reads in family Totiviridae,viruses
3,2603435,1,-200011006,Aedes aegypti totivirus,viruses
4,1884455,2,1884455,Anphevirus,viruses


## 3. Combine all the different files, keeping just the totCounts by merging with that taxa df. 

In [9]:
colnames=["tax_id",	"tax_level", 'genus_tax_id','name',	'category']
for key in dfs_nrnt:
    colnames.append(key)
    df_merged_cnts = pd.merge(df_merged_cnts, dfs_nrnt[key][['tax_id','totCounts']], on=['tax_id'], how='outer')


Put the new column names in

In [10]:
df_merged_cnts.columns=colnames
df_merged_cnts.head()

,tax_id,tax_level,genus_tax_id,name,category,SRR11440601,SRR11440602,SRR11440603,SRR11440604,SRR11440605,...,SRR11440735,SRR11440736,SRR11440737,SRR11440738,SRR11440739,SRR11440740,SRR11440741,SRR11440742,SRR11440743,SRR11440744
0,1980421,2,1980421,Phasivirus,viruses,1425.0,8477.0,7805.0,8229.0,8353.0,...,2582.0,3136.0,3071.0,2262.0,2121.0,2302.0,2293.0,1453.0,638.0,634.0
1,1980610,1,1980421,Phasi Charoen-like phasivirus,viruses,1425.0,8477.0,7805.0,8229.0,8353.0,...,2582.0,3136.0,3071.0,2262.0,2121.0,2302.0,2293.0,1453.0,638.0,634.0
2,-200011006,2,-200011006,non-genus-specific reads in family Totiviridae,viruses,244.0,16.0,12.0,20.0,17.0,...,10.0,18.0,8.0,NaN,NaN,NaN,NaN,4.0,4.0,89.0
3,2603435,1,-200011006,Aedes aegypti totivirus,viruses,244.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0
4,1884455,2,1884455,Anphevirus,viruses,80.0,204.0,378.0,259.0,202.0,...,461.0,646.0,580.0,424.0,418.0,518.0,373.0,40.0,102.0,120.0


## 4&5: Remove all eukaryota and then split into two files (level 1 taxa eg spp, and level 2 taxa eg genus). 

In [11]:
df_merged_cnts_notEuk1=df_merged_cnts.loc[(df_merged_cnts.category != 'eukaryota') & (df_merged_cnts.tax_level == 1)]
df_merged_cnts_notEuk2=df_merged_cnts.loc[(df_merged_cnts.category != 'eukaryota') & (df_merged_cnts.tax_level == 2)]

# df_merged_cnts_notEuk.head()
df_merged_cnts_notEuk1.to_csv('totalCounts_notEuk_taxLevel1.csv') # excluding all eukayotes (eg. mice but also fungus:( )
df_merged_cnts_notEuk2.to_csv('totalCounts_notEuk_taxLevel2.csv') # excluding all eukayotes (eg. mice but also fungus:( )